In [1]:
import os
from os.path import join

import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
inputPath = join("./","DataSet")
inputPath

'./DataSet'

In [3]:
os.listdir(inputPath)

['iris.csv']

In [4]:
# 데이터 불러들이기
iris = pd.read_csv(join(inputPath,'iris.csv'), names=("sepal length", "sepal width", "petal length", "petal width", "Class"))
iris.head()

,sepal length,sepal width,petal length,petal width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
#클래스 이름 확인
classes = iris["Class"].unique()

In [6]:
# 클래스 이름을 Iris-setosa : 1, Iris-versicolor : 2, Iris-virginica : 3 으로 대체
iris["Class"].replace(classes[0], 1, inplace = True) 
iris["Class"].replace(classes[1], 2, inplace = True) 
iris["Class"].replace(classes[2], 3, inplace = True) 

# 복원 추출로 10개를 샘플링해 제대로 클래스 이름이 숫자로 대체 되었는지 확인.
iris.sample(10)

,sepal length,sepal width,petal length,petal width,Class
83,6.0,2.7,5.1,1.6,2
26,5.0,3.4,1.6,0.4,1
132,6.4,2.8,5.6,2.2,3
50,7.0,3.2,4.7,1.4,2
18,5.7,3.8,1.7,0.3,1
134,6.1,2.6,5.6,1.4,3
30,4.8,3.1,1.6,0.2,1
89,5.5,2.5,4.0,1.3,2
45,4.8,3.0,1.4,0.3,1
94,5.6,2.7,4.2,1.3,2


One vs Rest Multiclass Classification https://en.wikipedia.org/wiki/Multiclass_classification#One-vs.-rest

선형 분리로 One vs Rest 다중 클래스 분류를 할것이기 때문에 SVM Classifier의 Kernel 중 LinearKernel을 사용합니다.

In [7]:
iris_x = deepcopy(iris)
iris_y = deepcopy(iris['Class'])
del iris_x['Class']

In [8]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(iris_x,iris_y, test_size=0.20)
# 훈련데이터와 테스트 데이터로 분리


In [9]:
# 분류기에 넣을 데이터 준비. 1번 분류기는 1번 클래스에 대해서만 True로 판별하고 2,3 번 클래스에 대해서는 False로 판별해야 하므로,
# 1번 클래스만 1로 2,3번 클래스는 0으로 변경하여 데이터를 넣어줄 예정. 2번 3번 분류기에 대해서도 동일하게 데이터를 넣어주려고 한다.
x_train = list()
y_train = list()
x_test = list()
y_test = list()
for i in range(len(classes)):
    x_train.append(deepcopy(train_x))
    y_train.append(deepcopy(train_y))
    x_test.append(deepcopy(test_x))
    y_test.append(deepcopy(test_y))
    if i == 0:
        y_train[i].replace(i,1, inplace = True)
        y_train[i].replace(2,0, inplace = True)
        y_train[i].replace(3,0, inplace = True)
        y_test[i].replace(i,1, inplace = True)
        y_test[i].replace(2,0, inplace = True)
        y_test[i].replace(3,0, inplace = True)
    elif i == 1:
        y_train[i].replace(1,0, inplace = True)
        y_train[i].replace(i,1, inplace = True)
        y_train[i].replace(3,0, inplace = True)
        y_test[i].replace(1,0, inplace = True)
        y_test[i].replace(i,1, inplace = True)
        y_test[i].replace(3,0, inplace = True)
    else :
        y_train[i].replace(1,0, inplace = True)
        y_train[i].replace(2,0, inplace = True)
        y_train[i].replace(i,1, inplace = True)
        y_test[i].replace(1,0, inplace = True)
        y_test[i].replace(2,0, inplace = True)
        y_test[i].replace(i,1, inplace = True)

In [61]:
from sklearn.neural_network import MLPClassifier
# Multi Layer Perceptron Classifier의 매개변수인 hidden_layer_size를 (1,)로 설정해 1개의 은닉층에 1개의 노드가 있도록하여
# 직선으로 분류를 하는 Perceptron으로 만들 수 있다.

# Class 1만을 분류할 수 있는 Perceptron
clf1 = MLPClassifier(hidden_layer_sizes=(1,));
clf1.fit(x_train[0], y_train[0]);

# Class 2만을 분류할 수 있는 Perceptron
clf2 = MLPClassifier(hidden_layer_sizes=(1,));
clf2.fit(x_train[1], y_train[1]);

# Class 3만을 분류할 수 있는 Perceptron
clf3 = MLPClassifier(hidden_layer_sizes=(1,));
clf3.fit(x_train[2], y_train[2]);



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [62]:
clf1.score(x_test[0], y_test[0])

0.5

In [63]:
clf2.score(x_test[1], y_test[1])

0.6666666666666666

In [64]:
clf3.score(x_test[2], y_test[2])

0.6

In [65]:
predict_y = list()

# Class 1, 2, 3 을 분류하는 분류기에 넣어 해당 Class일 확률을 구한다.
predict_y.append(clf1.predict_proba(test_x)[:,1])
predict_y.append(clf2.predict_proba(test_x)[:,1])
predict_y.append(clf3.predict_proba(test_x)[:,1])

# 확률이 가장 큰 Class를 선택한다. (0,1,2 로 출력되니 우리의 클래스 1,2,3에 맞게 1을 더한다.)
predict_y = np.argmax(predict_y, axis=0) + 1

In [66]:
# test_y 와 예측한 y의 값을 비교해 정확도를 얻어낸다.
accuracy = sum([ 1 if predict_y == true_y else 0 for predict_y, true_y in zip(predict_y, test_y)])/len(test_y)

print(accuracy)

0.5


생각보다 정확도가 잘 안나왔다. 아무래도 3개의 직선으로만 표현하려다 보니 가운데에 위치하거나 특정 위치의 값을 잘 분류하지 못하는 것 같다.